In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import functions as f

# Veriyi yükle
train = pd.read_parquet('train_final.parquet') 

In [2]:
#Veri ön işleme
train = train.drop(['month'], axis=1)

X_train_full = train.drop('target', axis=1)
y_train_full = train['target']

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

X_train_ids = X_train['id']
X_valid_ids = X_valid['id']

X_train = X_train.drop(['carrier', 'devicebrand'], axis=1)
X_valid = X_valid.drop(['carrier', 'devicebrand'], axis=1)

X_train = X_train.drop(['id'], axis=1)
X_valid = X_valid.drop(['id'], axis=1)

In [3]:
y_train = y_train.apply(f.target_to_binary)
y_valid = y_valid.apply(f.target_to_binary)

In [4]:
y_train = y_train.apply(f.map_binary_to_integer)
y_valid = y_valid.apply(f.map_binary_to_integer)

In [5]:
# CatBoost modelini oluştur
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=5000, learning_rate=0.1, depth=6, loss_function='MultiClass', eval_metric='Accuracy', random_seed=42, od_type='Iter', od_wait=100)


In [6]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=True, verbose=True)

0:	learn: 0.1783118	test: 0.1795853	best: 0.1795853 (0)	total: 2.95s	remaining: 4h 5m 55s
1:	learn: 0.1866186	test: 0.1859117	best: 0.1859117 (1)	total: 5.72s	remaining: 3h 58m 22s
2:	learn: 0.1918952	test: 0.1904838	best: 0.1904838 (2)	total: 8.42s	remaining: 3h 53m 39s
3:	learn: 0.1941546	test: 0.1919192	best: 0.1919192 (3)	total: 11.1s	remaining: 3h 50m 17s
4:	learn: 0.1955901	test: 0.1930356	best: 0.1930356 (4)	total: 13.6s	remaining: 3h 46m 57s
5:	learn: 0.1980223	test: 0.1953216	best: 0.1953216 (5)	total: 16.3s	remaining: 3h 46m 13s
6:	learn: 0.1994179	test: 0.1964381	best: 0.1964381 (6)	total: 19s	remaining: 3h 46m 7s
7:	learn: 0.2009596	test: 0.1989367	best: 0.1989367 (7)	total: 21.7s	remaining: 3h 46m
8:	learn: 0.2025014	test: 0.1993620	best: 0.1993620 (8)	total: 24.9s	remaining: 3h 49m 42s
9:	learn: 0.2039501	test: 0.2002127	best: 0.2002127 (9)	total: 28.3s	remaining: 3h 55m 28s
10:	learn: 0.2054121	test: 0.2022329	best: 0.2022329 (10)	total: 32.1s	remaining: 4h 2m 25s
11:	le

In [7]:
import pickle
pickle.dump(model, open("model.pickle.dat", "wb"))

In [8]:
y_pred = model.predict(X_valid)

In [9]:
jaccard = f.jaccard_similarity_score(y_valid, y_pred)
jaccard

0.6766183415606372

In [10]:
Test = pd.read_parquet('test_final.parquet') 

In [11]:
Test = Test.drop(['month','carrier','devicebrand'], axis=1)
Test_ids = Test['id']
Test = Test.drop(['id'], axis=1)

In [12]:
Test_y_pred = model.predict(Test)

In [13]:
Test_y_pred

array([[44],
       [ 3],
       [44],
       ...,
       [35],
       [37],
       [35]], dtype=int64)

In [17]:
def map_integer_to_binary(integer):
    #map_binary_to_integer fonksiyonunun tersi. 
    binary_numbers = f.generate_binary_numbers_with_three_ones()
    if integer < len(binary_numbers):
        return binary_numbers[integer]
    else:
        return None

In [ ]:
import numpy as np
np.ndindex(Test_y_pred)

In [ ]:
Test_y_pred = [f.target_to_binary(x) for x in Test_y_pred]

In [ ]:
ssf = pd.read_parquet('submission_sample_final.parquet') 

In [ ]:
tahmin_verisi = pd.DataFrame({'id': Test_ids, 'target': Test_y_pred})
tahmin_verisi.to_csv('XGB3.csv', index=False, columns=['id', 'target'])